In [5]:
import sys
sys.path.append('../')

from lxml import etree
import abc
import re
import requests

from gensim.models.phrases import *

import gensim.models as models
import gensim.models.word2vec as word2vec
import secure

import searchbetter.search as search
reload(search)
import searchbetter.rewriter as rewriter
reload(rewriter)


<module 'searchbetter.rewriter' from '../searchbetter/rewriter.pyc'>

# NEW CHECKPOINT 4/17 5:18p

In [72]:
import searchbetter.rewriter as rewriter 
reload(rewriter)
import secure

model_path = secure.MODEL_PATH_BASE+'word2vec/word2vec'
w2v_rewriter = rewriter.Word2VecRewriter(model_path, create=False)

In [75]:
w2v_rewriter.rewrite("New York")

[u'philadelphia',
 u'los angeles',
 u'london',
 u'boston',
 u'chicago',
 u'princeton',
 u'seattle',
 u'chicago illinois',
 u'san francisco',
 u'ny',
 u'New York']

In [87]:
## SEARCH ENGINE DEMOS
from pprint import pprint

import searchbetter.search as search
reload(search)

import secure

# Create a search engine that searches over all edX courses.
# Under the hood, this uses Python's Whoosh library to index
# the course data stored in a CSV and then run searches against it.
dataset_path = secure.DATASET_PATH_BASE+'Master CourseListings - edX.csv'
index_path = secure.INDEX_PATH_BASE+'edx'
edx_engine = search.EdXSearchEngine(dataset_path, index_path, create=False)

In [88]:
edx_engine.search("chemistry")

[{'course_id': u'DavidsonX/D.001x/3T2014', 'name': u'Medicinal Chemistry'},
 {'course_id': u'course-v1:DavidsonX+D001x+3T2015',
  'name': u'Medicinal Chemistry'},
 {'course_id': u'course-v1:CooperUnion+Chem.1x+2T2016',
  'name': u'Preparing for CLEP Chemistry: Part 1'},
 {'course_id': u'course-v1:CooperUnion+Chem.2x+2T2016',
  'name': u'Preparing for CLEP Chemistry: Part 2'},
 {'course_id': u'MITx/3.091x_2/1T2014',
  'name': u'Introduction to Solid State Chemistry'},
 {'course_id': u'MITx/3.091x_3/3T2014',
  'name': u'Introduction to Solid State Chemistry'},
 {'course_id': u'course-v1:MITx+3.091x_4+1T2015',
  'name': u'Introduction to Solid State Chemistry'},
 {'course_id': u'course-v1:MITx+3.091x_5+3T2015',
  'name': u'Introduction to Solid State Chemistry'},
 {'course_id': u'course-v1:MITx+3.091x+3T2016',
  'name': u'Introduction to Solid State Chemistry'},
 {'course_id': u'CooperUnion/Chem.1x/1T2015',
  'name': u'Preparing for the AP* Chemistry Exam - Part 1'}]

In [89]:
edx_engine.set_rewriter(w2v_rewriter)

In [90]:
edx_engine.search("chemistry")

[{'course_id': u'course-v1:HarvardX+MCB63X+3T2015',
  'name': u'Principles of Biochemistry'},
 {'course_id': u'MITx/7.28.1x/1T2015', 'name': u'Molecular Biology'},
 {'course_id': u'course-v1:RICEx+AdvBIOx+3T2016',
  'name': u'AP\xc2\xae Biology'},
 {'course_id': u'course-v1:HarvardX+MCB64.1x+2T2016',
  'name': u'Cell Biology: Mitochondria'},
 {'course_id': u'course-v1:IEEEx+SysBio1x+2016_T1',
  'name': u'Introduction to Systems Biology.'},
 {'course_id': u'course-v1:IEEEx+SysBio1x+3T2016',
  'name': u'Introduction to Systems Biology.'},
 {'course_id': u'MITx/7.QBWx/2T2014',
  'name': u'Quantitative Biology Workshop'},
 {'course_id': u'MITx/7.QBW_1x/1T2015',
  'name': u'Quantitative Biology Workshop'},
 {'course_id': u'course-v1:MITx+20.305x+3T2015',
  'name': u'Principles of Synthetic Biology'},
 {'course_id': u'course-v1:MITx+7.QBWx_3+1T2016',
  'name': u'Quantitative Biology Workshop'},
 {'course_id': u'course-v1:MITx+7.QBWx_4+3T2016',
  'name': u'Quantitative Biology Workshop'},
 {'

In [ ]:
qrse = search.RewritingSearchEngine()